### Import the Libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import missingno as msno

### Import the Train Dataset

In [2]:
df_train = pd.read_excel(r'Flight_Price_Train.xlsx')
df_train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


### Q1 Perform Feature Engineering 

#### a) Perform basic exploration like checking for top 5 records, shape, statistical info, duplicates, Null values etc. 

In [3]:
df_train.head(5)

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


In [4]:
df_train.shape

(10683, 11)

In [5]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          10683 non-null  object
 1   Date_of_Journey  10683 non-null  object
 2   Source           10683 non-null  object
 3   Destination      10683 non-null  object
 4   Route            10682 non-null  object
 5   Dep_Time         10683 non-null  object
 6   Arrival_Time     10683 non-null  object
 7   Duration         10683 non-null  object
 8   Total_Stops      10682 non-null  object
 9   Additional_Info  10683 non-null  object
 10  Price            10683 non-null  int64 
dtypes: int64(1), object(10)
memory usage: 918.2+ KB


In [6]:
df_train.duplicated().sum()

220

In [7]:
df_train.isna().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              1
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        1
Additional_Info    0
Price              0
dtype: int64

#### b) Extract Date, Month, Year from  Date of Journey column

In [8]:
df_train['Date_of_Journey'] = pd.to_datetime(df_train['Date_of_Journey'], format='%d/%m/%Y')
df_train['Journey_Date'] = df_train['Date_of_Journey'].dt.day
df_train['Journey_Month'] = df_train['Date_of_Journey'].dt.month
df_train['Journey_Year'] = df_train['Date_of_Journey'].dt.year

df_train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Journey_Date,Journey_Month,Journey_Year
0,IndiGo,2019-03-24,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897,24,3,2019
1,Air India,2019-05-01,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662,1,5,2019
2,Jet Airways,2019-06-09,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882,9,6,2019
3,IndiGo,2019-05-12,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218,12,5,2019
4,IndiGo,2019-03-01,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302,1,3,2019


### Q2 Perform Exploratory Data Analysis (EDA) tasks

#### a) Which airline is most preferred airline

In [9]:
most_preferred_airline = df_train['Airline'].value_counts().idxmax()
print("Most preferred airline:", most_preferred_airline)

Most preferred airline: Jet Airways


#### b) Find the majority of the flights take off from which source

In [10]:
majority_source = df_train['Source'].value_counts().idxmax()
print("Majority of flights take off from:", majority_source)

Majority of flights take off from: Delhi


#### c) Find maximum flights land in which destination

In [11]:
max_land_destination = df_train['Destination'].value_counts().idxmax()
print("Maximum flights land in destination:", max_land_destination)

Maximum flights land in destination: Cochin


### Q3 Compare independent features with Target feature to check the impact on price

#### a) Which airline has the highest price 

In [12]:
airline_highest_price = df_train.loc[df_train['Price'].idxmax(), 'Airline']
highest_price = df_train['Price'].max()
print("Airline with the highest price:", airline_highest_price)
print("Highest price:", highest_price)

Airline with the highest price: Jet Airways Business
Highest price: 79512


#### b) Check if the business class flights are high price or low and find only those flights which price is higher than 50k


In [13]:
business_class_flights = df_train[df_train['Additional_Info'].str.contains('Business', case=False)]
high_price_business_flights = business_class_flights[business_class_flights['Price'] > 50000]
print("\nBusiness class flights with price higher than 50k:")
print(high_price_business_flights)


Business class flights with price higher than 50k:
                    Airline Date_of_Journey    Source Destination  \
2924   Jet Airways Business      2019-03-01  Banglore   New Delhi   
5372   Jet Airways Business      2019-03-01  Banglore   New Delhi   
10364  Jet Airways Business      2019-03-01  Banglore   New Delhi   

                 Route Dep_Time Arrival_Time Duration Total_Stops  \
2924   BLR → BOM → DEL    05:45        11:25   5h 40m      1 stop   
5372   BLR → BOM → DEL    05:45        12:25   6h 40m      1 stop   
10364  BLR → MAA → DEL    09:45        14:25   4h 40m      1 stop   

      Additional_Info  Price  Journey_Date  Journey_Month  Journey_Year  
2924   Business class  79512             1              3          2019  
5372   Business class  62427             1              3          2019  
10364  Business class  57209             1              3          2019  


### Q4 Perform encoding for the required features according to the data.

#### Calling the columns having dtype as object

In [14]:
colname=[]
for x in df_train.columns:
    if df_train[x].dtype=='object':
        colname.append(x)
colname

['Airline',
 'Source',
 'Destination',
 'Route',
 'Dep_Time',
 'Arrival_Time',
 'Duration',
 'Total_Stops',
 'Additional_Info']

#### Converting Categorical Variables into Numerical Variables by using LabelEncoder()

In [15]:
from sklearn.preprocessing import LabelEncoder
 
le=LabelEncoder()
 
for x in colname:
    df_train[x]=le.fit_transform(df_train[x])

In [16]:
df_train.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Journey_Date,Journey_Month,Journey_Year
0,3,2019-03-24,0,5,18,211,233,240,4,8,3897,24,3,2019
1,1,2019-05-01,3,0,84,31,906,336,1,8,7662,1,5,2019
2,4,2019-06-09,2,1,118,70,413,106,1,8,13882,9,6,2019
3,3,2019-05-12,3,0,91,164,1324,311,0,8,6218,12,5,2019
4,3,2019-03-01,0,5,29,149,1237,303,0,8,13302,1,3,2019


In [17]:
df_train.dropna(inplace=True)

In [18]:
df_train.shape

(10683, 14)

### Q5	Build multiple model by using different algorithm such as Linear Regression, Decision Tree, and Random                 Forest etc. and check the performance of your model.	

#### Dropping the null values

In [19]:
df_train.isna().sum()

Airline            0
Date_of_Journey    0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
Price              0
Journey_Date       0
Journey_Month      0
Journey_Year       0
dtype: int64

#### Dropping the Duplicates Value

In [20]:
df_train.drop_duplicates(inplace=True)

In [21]:
df_train.duplicated().sum()

0

#### Dropping the Date_of_Journey Column

In [22]:
df_train.drop(columns='Date_of_Journey', inplace=True)
df_train.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price,Journey_Date,Journey_Month,Journey_Year
0,3,0,5,18,211,233,240,4,8,3897,24,3,2019
1,1,3,0,84,31,906,336,1,8,7662,1,5,2019
2,4,2,1,118,70,413,106,1,8,13882,9,6,2019
3,3,3,0,91,164,1324,311,0,8,6218,12,5,2019
4,3,0,5,29,149,1237,303,0,8,13302,1,3,2019


#### Create X and Y

In [23]:
X =df_train.drop(columns='Price')
Y = df_train['Price']

In [24]:
print(X.shape)
print(Y.shape)
Y=Y.astype(int)

(10463, 12)
(10463,)


### Standardization

In [25]:
from sklearn.preprocessing import StandardScaler

scaler= StandardScaler()

scaler.fit(X)
X= scaler.transform(X)

#### Split the data into training and testing sets

In [26]:
from sklearn.model_selection import train_test_split
 
#Split the data into test and train
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, 
                                                    random_state=10)

#### Building the Model

In [27]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [28]:
# Initialize models
linear_regression = LinearRegression()
decision_tree = DecisionTreeRegressor(random_state=10)
random_forest = RandomForestRegressor(random_state=10)

In [29]:
# Train the models
linear_regression.fit(X_train, Y_train)
decision_tree.fit(X_train, Y_train)
random_forest.fit(X_train, Y_train)

RandomForestRegressor(random_state=10)

In [30]:
# Predict on the testing set
y_pred_lr = linear_regression.predict(X_test)
y_pred_dt = decision_tree.predict(X_test)
y_pred_rf = random_forest.predict(X_test)

In [31]:
# Evaluate model performance
def evaluate_model(y_true, y_pred):
    rmse=np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    adjusted_r_squared = 1 - (1-r2)*(len(Y)-1)/(len(Y)-X.shape[1]-1)
    return rmse, mae, r2, adjusted_r_squared

print("Linear Regression:")
rmse_lr, mae_lr, r2_lr, adjusted_r_squared_lr = evaluate_model(Y_test, y_pred_lr)
print("RMSE:", rmse_lr)
print("Mean Absolute Error:", mae_lr)
print("R^2 Score:", r2_lr)
print("Adj R-square:",adjusted_r_squared_lr)

print("\nDecision Tree:")
rmse_dt, mae_dt, r2_dt, adjusted_r_squared_dt = evaluate_model(Y_test, y_pred_dt)
print("RMSE:", rmse_dt)
print("Mean Absolute Error:", mae_dt)
print("R^2 Score:", r2_dt)
print("Adj R-square:",adjusted_r_squared_dt)

print("\nRandom Forest:")
rmse_rf, mae_rf, r2_rf, adjusted_r_squared_rf = evaluate_model(Y_test, y_pred_rf)
print("RMSE:", rmse_rf)
print("Mean Absolute Error:", mae_rf)
print("R^2 Score:", r2_rf)
print("Adj R-square:",adjusted_r_squared_rf)

Linear Regression:
RMSE: 3588.996445027715
Mean Absolute Error: 2631.322446531292
R^2 Score: 0.36815048789921
Adj R-square: 0.36742491908148667

Decision Tree:
RMSE: 2252.3261153754975
Mean Absolute Error: 733.8108739513644
R^2 Score: 0.7511542986202399
Adj R-square: 0.7508685427909043

Random Forest:
RMSE: 1525.5596333255369
Mean Absolute Error: 663.0395091349485
R^2 Score: 0.885836841560582
Adj R-square: 0.8857057451106993


### Note: From the result, Random Forest gave the least RMSE compare to both Linear Regression and Decision Tree. So we will further continue to Optimize Random Forest inorder to increase the efficiency of the model.

In [32]:
# Hyperparameter tuning for Random Forest
from sklearn.model_selection import GridSearchCV
param_grid_rf = {
    'n_estimators': [100, 200, 300],
    #'max_depth': [None, 10, 20, 30],
    #'min_samples_split': [2, 5, 10],
    #'min_samples_leaf': [1, 2, 4]
}

grid_search_rf = GridSearchCV(estimator=RandomForestRegressor(random_state=42),
                              param_grid=param_grid_rf,
                              scoring='neg_mean_squared_error',
                              cv=5,
                              n_jobs=-1)

grid_search_rf.fit(X_train, Y_train)
best_params_rf = grid_search_rf.best_params_
best_estimator_rf = grid_search_rf.best_estimator_
print("Best parameters for Random Forest:", best_params_rf)

Best parameters for Random Forest: {'n_estimators': 200}


In [33]:
grid_search_rf.best_score_ 

-2856154.288802453

In [34]:
Y_pred=grid_search_rf.predict(X_test)

In [35]:
from sklearn.metrics import r2_score,mean_squared_error
import numpy as np

r2=r2_score(Y_test,Y_pred)
print("R-squared:",r2)

rmse=np.sqrt(mean_squared_error(Y_test,Y_pred))
print("RMSE:",rmse)

adjusted_r_squared = 1 - (1-r2)*(len(Y)-1)/(len(Y)-X.shape[1]-1)
print("Adj R-square:",adjusted_r_squared)

R-squared: 0.8835624160040338
RMSE: 1540.6812373127054
Adj R-square: 0.8834287077736078


#### In the pruning of Random Forest we can use multiple hyperparameter as per the requirement. While for me it was taking too long for computation so I used only one i.e. n_estimators

### Q.7	Write a conclusion from the business point of view. Finally perform the same preprocessing technique for test data best practice using pipeline.

#### 1) Most Preferred Airline: Jet Airways
    Jet Airways is the top choice for most passengers, indicating its popularity and good service.
    Business Impact: Jet Airways should keep up their good service and focus on marketing to maintain their lead.

#### 2) Majority of Flights Take Off from Delhi
    Delhi is the busiest departure point for flights, showing high demand.
    Business Impact: Airlines should manage resources well and consider expanding in Delhi to meet demand.

#### 3) Maximum Flights Land in Destination: Cochin
    Cochin is the most common destination for flights, showing its popularity.
    Business Impact: Airlines should be ready to handle the high volume of flights landing in Cochin and offer good services.

#### 4) Highest Price Airline: Jet Airways Business
    Jet Airways Business has the highest ticket prices, likely due to its premium offerings.
    Business Impact: Jet Airways Business should maintain its premium services to justify the higher prices.

####    In conclusion, understanding customer preferences and market trends is key for airlines to make informed decisions and stay    competitive.

## Test Data

In [36]:
df_test = pd.read_excel(r'Flight_Price_Test.xlsx')
df_test.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


In [37]:
df_test['Date_of_Journey'] = pd.to_datetime(df_test['Date_of_Journey'], format='%d/%m/%Y')
df_test['Journey_Date'] = df_test['Date_of_Journey'].dt.day
df_test['Journey_Month'] = df_test['Date_of_Journey'].dt.month
df_test['Journey_Year'] = df_test['Date_of_Journey'].dt.year

df_test.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Journey_Date,Journey_Month,Journey_Year
0,Jet Airways,2019-06-06,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info,6,6,2019
1,IndiGo,2019-05-12,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info,12,5,2019
2,Jet Airways,2019-05-21,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included,21,5,2019
3,Multiple carriers,2019-05-21,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info,21,5,2019
4,Air Asia,2019-06-24,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info,24,6,2019


In [38]:
df_test.drop(columns='Date_of_Journey', inplace=True)
df_test.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Journey_Date,Journey_Month,Journey_Year
0,Jet Airways,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info,6,6,2019
1,IndiGo,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info,12,5,2019
2,Jet Airways,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included,21,5,2019
3,Multiple carriers,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info,21,5,2019
4,Air Asia,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info,24,6,2019


In [39]:
df_test.shape

(2671, 12)

In [40]:
df_test.duplicated().sum()

26

In [41]:
df_test.drop_duplicates(inplace=True)

In [42]:
df_test.duplicated().sum()

0

In [43]:
df_test.isna().sum()

Airline            0
Source             0
Destination        0
Route              0
Dep_Time           0
Arrival_Time       0
Duration           0
Total_Stops        0
Additional_Info    0
Journey_Date       0
Journey_Month      0
Journey_Year       0
dtype: int64

In [44]:
df_test.describe(include='all')

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Journey_Date,Journey_Month,Journey_Year
count,2645,2645,2645,2645,2645,2645,2645,2645,2645,2645.000000,2645.000000,2645.0
unique,11,5,6,100,199,704,320,5,6,NaN,NaN,NaN
top,Jet Airways,Delhi,Cochin,DEL → BOM → COK,10:00,19:00,2h 50m,1 stop,No info,NaN,NaN,NaN
freq,886,1127,1127,620,61,112,121,1427,2126,NaN,NaN,NaN
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.908129,4.715690,2019.0
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.267572,1.170134,0.0
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,3.000000,2019.0
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6.000000,3.000000,2019.0
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.000000,5.000000,2019.0
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,21.000000,6.000000,2019.0


In [45]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2645 entries, 0 to 2670
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Airline          2645 non-null   object
 1   Source           2645 non-null   object
 2   Destination      2645 non-null   object
 3   Route            2645 non-null   object
 4   Dep_Time         2645 non-null   object
 5   Arrival_Time     2645 non-null   object
 6   Duration         2645 non-null   object
 7   Total_Stops      2645 non-null   object
 8   Additional_Info  2645 non-null   object
 9   Journey_Date     2645 non-null   int32 
 10  Journey_Month    2645 non-null   int32 
 11  Journey_Year     2645 non-null   int32 
dtypes: int32(3), object(9)
memory usage: 237.6+ KB


In [46]:
colname=[]
for x in df_test.columns:
    if df_test[x].dtype=='object':
        colname.append(x)
colname

['Airline',
 'Source',
 'Destination',
 'Route',
 'Dep_Time',
 'Arrival_Time',
 'Duration',
 'Total_Stops',
 'Additional_Info']

#### Converting Categorical Variables into Numerical Variables by using LabelEncoder()

In [47]:
from sklearn.preprocessing import LabelEncoder
 
le=LabelEncoder()
 
for x in colname:
    df_test[x]=le.fit_transform(df_test[x])

#### Create X

In [48]:
X_test_new = df_test

In [49]:
print(X_test_new.shape)

(2645, 12)


In [50]:
X_test_new = scaler.transform(X_test_new)
#X=scaler.fit_transform(X)
#print(X)

In [51]:
print(X_test_new)

[[ 0.0081243   0.03907712 -0.29675162 ... -0.88136161  1.11582544
   0.        ]
 [-0.41478375  0.88209553 -0.97202613 ... -0.17272791  0.25649366
   0.        ]
 [ 0.0081243   0.03907712 -0.29675162 ...  0.89022263  0.25649366
   0.        ]
 ...
 [ 0.0081243   0.03907712 -0.29675162 ... -0.88136161 -1.46216991
   0.        ]
 [-1.26059986  0.03907712 -0.29675162 ... -0.88136161 -1.46216991
   0.        ]
 [ 0.8539404   0.03907712 -0.29675162 ...  0.18158894  1.11582544
   0.        ]]


In [52]:
Y_pred_new=random_forest.predict(X_test_new)
print(Y_pred_new)

[10003.07   4957.09  28237.96  ... 13210.05  14088.89   9721.465]


In [53]:
#df_test = pd.read_excel(r'Flight_Price_Test.xlsx',header=0)
df_test["Pred"]=Y_pred_new
df_test.head()

,Airline,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Journey_Date,Journey_Month,Journey_Year,Pred
0,4,2,1,76,138,150,10,0,5,6,6,2019,10003.07
1,3,3,0,65,31,353,248,0,5,12,5,2019,4957.09
2,4,2,1,76,156,566,148,0,3,21,5,2019,28237.96
3,6,2,1,76,48,633,36,0,5,21,5,2019,9423.22
4,0,0,2,16,198,135,216,4,5,24,6,2019,4235.92


In [54]:
df_test.Pred.value_counts()

Pred
4233.780     3
10003.070    2
4476.650     2
25413.770    2
4000.010     2
            ..
3972.250     1
4000.850     1
4053.930     1
13048.410    1
9721.465     1
Name: count, Length: 2589, dtype: int64

In [55]:
#df_test.to_excel('Flight Test RandomForest Output.xlsx',header=True)

### Conclusion: The Random Forest Regressor gave the least RMSE and best Adjusted R^2. Therefore it was use for test data prediction.